### Notebook to explore algorithmic data segmentations 
* For use with supervised models or in evaluation of unsupervised models
* Segmentations computed using "personal" segmentation algorithm contributed to SunPy

#### Segment DKIST

In [ ]:
'''
Segment DKIST VBI_2022_05 series [Oct 28, 2023]
'''
res = 0.016
pad = 80 # 80 for Oct 28, need more padding than for others (bigger boarders)
footprint = 200 #30
files = [filename for filename in os.listdir(dir) if filename.startswith('VBI')]
for file in files:
    print(f'Input file {str(file)}')
    savepath = f'{dir}SEGv2_{file}'
    if os.path.exists(savepath):
        print(f'\t{savepath} Segmented data already exists')
    else: 
        data = fits.open(dir+file)[0].data 
        header = fits.open(dir+file)[0].header
        seg_data = funclib.segment_array_v2(data, resolution=res, mark_dim_centers=True, bp_max_size=0.18, footprint=footprint, pad=pad) 
        seg_hdu = fits.PrimaryHDU(seg_data)
        raw_hdu = fits.ImageHDU(data)
        hdu = fits.HDUList([seg_hdu, raw_hdu])
        print(f'\tWriting segmented data to '+str(savepath))
        hdu.writeto(savepath, overwrite=True)
print(f'Done all {len(files)} files'); a=b

In [ ]:
'''
Check full segmented Oct 28 images
 - One bad one... idk what to do with that. Should I throw it out now? Maybe but im not going to.
'''
files = [file for file in os.listdir(dir) if file.startswith('SEG')]
n = int((np.floor(np.sqrt(len(files)/4))))
idx = 0
fig, axs = plt.subplots(n, n, figsize=(80,80))
for i in range(n):
    for j in range(n):
        if idx < n**2:
            axs[i,j].imshow(fits.open(f'{dir}{files[idx]}')[0].data)
            axs[i,j].set_title(files[idx])
            idx += 1
        axs[i,j].xaxis.set_tick_params(labelbottom=False); axs[i,j].yaxis.set_tick_params(labelleft=False); axs[i,j].set_xticks([]); axs[i,j].set_yticks([])
plt.savefig('DISKT_series_Oct28_segs_a'); print('Saved fig')
fig, axs = plt.subplots(n, n, figsize=(20,20))
for i in range(n):
    for j in range(n):
        if idx < 2*n**2:
            axs[i,j].imshow(fits.open(f'{dir}{files[idx]}')[0].data)
            axs[i,j].set_title(files[idx])
            idx += 1
        axs[i,j].xaxis.set_tick_params(labelbottom=False); axs[i,j].yaxis.set_tick_params(labelleft=False); axs[i,j].set_xticks([]); axs[i,j].set_yticks([])
plt.savefig('DISKT_series_Oct28_segs_b'); print('Saved fig')
fig, axs = plt.subplots(n, n, figsize=(20,20))
for i in range(n):
    for j in range(n):
        if idx < 3*n**2:
            axs[i,j].imshow(fits.open(f'{dir}{files[idx]}')[0].data)
            axs[i,j].set_title(files[idx])
            idx += 1
        axs[i,j].xaxis.set_tick_params(labelbottom=False); axs[i,j].yaxis.set_tick_params(labelleft=False); axs[i,j].set_xticks([]); axs[i,j].set_yticks([])
plt.savefig('DISKT_series_Oct28_segs_c'); print('Saved fig')
fig, axs = plt.subplots(n, n, figsize=(20,20))
for i in range(n):
    for j in range(n):
        if idx < 4*n**2:
            axs[i,j].imshow(fits.open(f'{dir}{files[idx]}')[0].data)
            axs[i,j].set_title(files[idx])
            idx += 1
        axs[i,j].xaxis.set_tick_params(labelbottom=False); axs[i,j].yaxis.set_tick_params(labelleft=False); axs[i,j].set_xticks([]); axs[i,j].set_yticks([])
plt.savefig('DISKT_series_Oct28_segs_d'); print('Saved fig')
bad = ['37_46_360']

#### Are adjustments needed to segment real (non FL) DKIST?
NOTE (7/14): is this how I ended up making segv2?

In [ ]:
'''
Segment DKIST 2022_06_03 mosaic [Jun 16, 2023]
    Ignore blurry ones which definitely wont be good. 
    For spiky ones, segment on corner region removed, then add padding of zero to keep size the same
'''

dir = '../Data/All_DKIST/'
skip = ['VBI_59_14_4096']
spike = ['VBI_31_21_4096', 'VBI_33_13_4096']
files = [filename for filename in os.listdir(dir) if filename.startswith('VBI') and filename.endswith('_4096')] 
for file in files:
    print(f'Input file {str(file)}')
    savepath = f'{dir}SEG_{file}'
    if os.path.exists(savepath):
        print(f'\t{savepath} Segmented data already exists')
    else:
        if any(tag in file for tag in skip): 
            print(f'\tSkipping {tag} (gets stuck on this file)') # WHY??
            continue 
        data = fits.open(dir+file)[1].data # for VBI files, data in HDU 1, for HM VBI files its in HDU 0
        header = fits.open(dir+file)[1].header
        if any(tag in file for tag in spike):
            cutdata = data[0:data.shape[1]-300, 0:data.shape[1]-300] 
            seg_datacut = funclib.segment_array(cutdata, resolution=header['cdelt1'], mark_dim_centers=True)
            seg_data = np.zeros_like(data) * np.NaN
            seg_data[0:4096-300, 0:4096-300] = seg_datacut
        else:
            seg_data = funclib.segment_array(data, resolution=header['cdelt1'], mark_dim_centers=True) #resolution=0.016)
        seg_hdu = fits.PrimaryHDU(seg_data)
        raw_hdu = fits.ImageHDU(data)
        hdu = fits.HDUList([seg_hdu, raw_hdu])
        print(f'\tWriting segmented data to '+str(savepath))
        hdu.writeto(savepath, overwrite=True)

In [ ]:
'''
Examine Jun16 segmentations
    OOOY these are crappy.
    Most of them do have BP just very very very few (see fits.open('../Data/All_DKIST/SEG_VBI_04_05_4096')[0].data[0:1000, 0:1000]).
    Also too much IG.
'''
dir = '../Data/All_DKIST/'
jun16tags = ['17_20', '17_23', '24_08', '24_23', '24_26', '25_39', '28_27', '31_21', '36_25', '04_05', '04_08', '04_11', '04_14', '04_17', '04_20', '04_23', '04_26', '04_29', '15_12', '15_15', '23_31', '23_34', '30_14', '31_27', '31_33', '32_52', '33_13', '37_15', '42_22', '42_31', '42_34', '46_32', '56_11', '56_20', '56_23', '59_14']
files = [filename for filename in os.listdir(dir) if filename.startswith('SEG_VBI') and filename.endswith('_4096') and any(tag in filename for tag in jun16tags)] 
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10), (ax11, ax12), (ax13, ax14), (ax15, ax16), (ax17, ax18), (ax19, ax20), (ax21, ax22), (ax23, ax24), (ax25, ax26), (ax27, ax28), (ax29, ax30), (ax31, ax32), (ax33, ax34), (ax35, ax36)) =  plt.subplots(18, 2, figsize=(25, 200)); axs = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12, ax13, ax14, ax15, ax16, ax17, ax18, ax19, ax20, ax21, ax22, ax23, ax24, ax25, ax26, ax27, ax28, ax29, ax30, ax31, ax32, ax33, ax34, ax35, ax36]
for i in range(0, len(files)):
    segdata = fits.open(dir+files[i])[0].data
    axs[i].set_title(files[i])
    #im = axs[i].imshow(segdata, origin='lower')
    justbp = np.zeros_like(segdata)*np.NaN
    justbp[segdata == 1.5] = 1
    im = axs[i].imshow(justbp, origin='lower')
    plt.colorbar(im, ax=axs[i])

In [ ]:
'''
Perhaps I could try adjusting thresholds? Or actually maybe HM would help.
    No.. that gets rid of the too much IG, but makes too much GR
'''

data = fits.open('../Data/All_DKIST/VBI_04_05_4096')[1].data
header = fits.open('../Data/All_DKIST/VBI_04_05_4096')[1].header
HMdata = funclib.match_to_firstlight(data)
hdu = fits.PrimaryHDU(HMdata, header=header)
hdu.writeto('../Data/All_DKIST/HM_VBI_04_05_4096', overwrite=True)
segHMdata = funclib.segment_array(HMdata, resolution=header['cdelt1'], mark_dim_centers=True), #resolution=0.016)
seg_hdu = fits.PrimaryHDU(segHMdata)
raw_hdu = fits.ImageHDU(data)
hdu = fits.HDUList([seg_hdu, raw_hdu])
hdu.writeto('../Data/All_DKIST/SEG_HM_VBI_04_05_4096', overwrite=True)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5))
data = fits.open('../Data/All_DKIST/VBI_04_05_4096')[1].data
im1 = ax1.imshow(data, origin='lower')
ax1.set_title('Image')
plt.colorbar(im1, ax=ax1)
segdata = fits.open('../Data/All_DKIST/SEG_VBI_04_05_4096')[0].data
im2 = ax2.imshow(np.squeeze(segdata), origin='lower')
ax2.set_title('Seg without HM')
plt.colorbar(im2, ax=ax2)
segHMdata = fits.open('../Data/All_DKIST/SEG_HM_VBI_04_05_4096')[0].data
im3 = ax3.imshow(np.squeeze(segHMdata), origin='lower')
ax3.set_title('Seg with HM')
plt.colorbar(im3, ax=ax3)

In [ ]:
'''
Lets try to adjust bp threshold? original is np.nanmean(data) + 0.5 * np.nanstd(data)
    BPL1: fac_brightness_limit = np.nanmean(data) + 0.25 * np.nanstd(data) --> very little change 
    BPL2: no fac_brightness_limit --> still very little change
    BPSL1: fac_pix_limit = 500 (for all VBI files, cdelt1 is 0.01099 so pix limit defualts to 181.8) --> again negligable change
    BPSL2: no pix size limit (fac_pix_limit = 100000000) --> yep, good. Marks lot of gr as fac as expected
    BPSL3: fac_pix_limit = 1000 --> much more fac! But how many do I want??
'''

data = fits.open('../Data/All_DKIST/VBI_04_05_4096')[1].data
header = fits.open('../Data/All_DKIST/VBI_04_05_4096')[1].header
segBPSL3data = funclib.segment_array(data, resolution=header['cdelt1'], mark_dim_centers=True, fac_pix_limit = 1000) #resolution=0.016)
seg_hdu = fits.PrimaryHDU(segBPSL3data)
raw_hdu = fits.ImageHDU(data)
hdu = fits.HDUList([seg_hdu, raw_hdu])
hdu.writeto('../Data/All_DKIST/SEGBPSL3_VBI_04_05_4096', overwrite=True)

HMdata = fits.open('../Data/All_DKIST/HM_VBI_04_05_4096')[0].data
segBPSL3HMdata = funclib.segment_array(HMdata, resolution=header['cdelt1'], mark_dim_centers=True, fac_pix_limit = 1000) #resolution=0.016)
seg_hdu = fits.PrimaryHDU(segBPSL3HMdata)
raw_hdu = fits.ImageHDU(data)
hdu = fits.HDUList([seg_hdu, raw_hdu])
hdu.writeto('../Data/All_DKIST/SEGBPSL3_HM_VBI_04_05_4096', overwrite=True)

fig, (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8) = plt.subplots(8, 1, figsize=(5, 40))
data = fits.open('../Data/All_DKIST/VBI_04_05_4096')[1].data
im1 = ax1.imshow(data, origin='lower')
ax1.set_title('Image')
plt.colorbar(im1, ax=ax1)
segdata = fits.open('../Data/All_DKIST/SEG_VBI_04_05_4096')[0].data
im2 = ax2.imshow(np.squeeze(segdata), origin='lower')
ax2.set_title('Seg without HM')
plt.colorbar(im2, ax=ax2)
segBPL2data = fits.open('../Data/All_DKIST/SEGBPL2_VBI_04_05_4096')[0].data
im3 = ax3.imshow(np.squeeze(segBPL2data), origin='lower')
ax3.set_title('Seg without HM, no brightness threshold')
plt.colorbar(im3, ax=ax3)
segBPSL3data = fits.open('../Data/All_DKIST/SEGBPSL3_VBI_04_05_4096')[0].data
im4 = ax4.imshow(np.squeeze(segBPSL3data), origin='lower')
ax4.set_title('Seg without HM, pix limit 1000')
plt.colorbar(im4, ax=ax4)

HMdata = fits.open('../Data/All_DKIST/HM_VBI_04_05_4096')[0].data
im5 = ax5.imshow(np.squeeze(HMdata), origin='lower')
ax5.set_title('HM Image')
plt.colorbar(im5, ax=ax5)
segHMdata = fits.open('../Data/All_DKIST/SEG_HM_VBI_04_05_4096')[0].data
im6 = ax6.imshow(np.squeeze(segHMdata), origin='lower')
ax6.set_title('Seg with HM')
plt.colorbar(im6, ax=ax6)
segBPL2HMdata = fits.open('../Data/All_DKIST/SEGBPL2_HM_VBI_04_05_4096')[0].data
im7 = ax7.imshow(np.squeeze(segBPL2HMdata), origin='lower')
ax7.set_title('Seg with HM, no brightness limit')
plt.colorbar(im7, ax=ax7)
segBPSL3HMdata = fits.open('../Data/All_DKIST/SEGBPSL3_HM_VBI_04_05_4096')[0].data
im8 = ax8.imshow(np.squeeze(segBPSL3HMdata), origin='lower')
ax8.set_title('Seg with HM, pix limit 100')
plt.colorbar(im8, ax=ax8)

'''
Ok, this shows a few things.
Main issue is all non-firstlight images have both varying res and more importantly varying overall brightness
Unfortunately, histogram matching doesnt really sovle the varying brightness; its still there
Obvioulsy my seg algo just doenst work with varying brightness, since it applies a uniform initial threhsold
Thus, it will allways ether overclassify IG in dim regions, or underclassify IG in bright regions, depending on pixel dist
I realy dont know how I would make there be a spatially-varying threshold
Another issue is that regions with lots of bp which I want) don't get segmented into granules with bp on top, but instead just weird, erratic granules
This means that it is going to be really really hard to get good segmentations that contain lots of bp
But maybe there will still be pieces I can pick out that do have "good" results.
Need to figure out how to get, if not actually good segmentations, segmentations that result in many good pieces
Ok, so using higher pix limit (allowing bigger things to be bp works).. but why???
Should change the fac_pix_limit = fac_size_limit / resolution, with fac_size_limit = 2"/pix??
'''

#### MURaM

In [ ]:
'''
Perform segmentations
'''

dir = '../Data/MURaM/'
res = 0.0211
imgfiles = [filename for filename in os.listdir(dir) if filename.startswith('I_out')]
print(f"Currently have segmented {len([filename for filename in os.listdir(dir) if filename.startswith('SEGv2')])} out of {len(imgfiles)} MURaM files")
files = np.random.choice(imgfiles, size = 1801) # Do all of them! ['I_out.010890.fits.gz']
n_added = 0
for file in files:
    print(f'Input file {str(file)}')
    savepath = f'{dir}SEGv2_{file}'
    if os.path.exists(savepath):
        print(f'\t{savepath} Segmented data already exists')
    else: 
        data = fits.open(dir+file)[0].data 
        header = fits.open(dir+file)[0].header
        seg_data = funclib.segment_array_v2(data, resolution=res, mark_dim_centers=True, bp_max_size=0.18, footprint=30) #resolution=0.016)
        seg_hdu = fits.PrimaryHDU(seg_data)
        raw_hdu = fits.ImageHDU(data)
        hdu = fits.HDUList([seg_hdu, raw_hdu])
        print(f'\tWriting segmented data to '+str(savepath))
        hdu.writeto(savepath, overwrite=True)
        n_added += 1
print(f'Done all {len(files)} files. Added {n_added} segs.')
print(f"Currently have segmented {len([filename for filename in os.listdir(dir) if filename.startswith('SEGv2')])} out of {len(imgfiles)} MURaM files")


#### Convert back from 4-class to 3 and 2

In [ ]:
'''
Create UNetData_v2_subset/seg_images_binary anologous to UNetData_v2_subset/seg_images
'''

# dir = "../Data/UNetData_v2_subset/"
# for file in os.listdir(f'{dir}seg_images/train/'):
#     if os.path.exists(f'{dir}seg_images_binary/train/{file}') == False:
#         seg = np.load(f'{dir}seg_images/train/{file}')
#         binseg = funclib.convert_back(seg, to='binary')
#         np.save(f'{dir}seg_images_binary/train/{file}', binseg)
# for file in os.listdir(f'{dir}seg_images/val/'):
#     if os.path.exists(f'{dir}seg_images_binary/val/{file}') == False:
#         seg = np.load(f'{dir}seg_images/val/{file}')
#         binseg = funclib.convert_back(seg, to='binary')
#         np.save(f'{dir}seg_images_binary/val/{file}', binseg)

'''
Going back to using just first light, but lets also do it on only 3 classes. 
Create UNetData_v2_subset/seg_images_ternary anologous to UNetData_v2_subset/seg_images
'''

# dir = "../Data/UNetData/"
# for file in os.listdir(f'{dir}seg_images/train/'):
#     if os.path.exists(f'{dir}seg_images_ternary/train/{file}') == False:
#         seg = np.load(f'{dir}seg_images/train/{file}')
#         triseg = funclib.convert_back(seg, to='ternary')
#         np.save(f'{dir}seg_images_ternary/train/{file}', triseg)
# for file in os.listdir(f'{dir}seg_images/val/'):
#     if os.path.exists(f'{dir}seg_images_ternary/val/{file}') == False:
#         seg = np.load(f'{dir}seg_images/val/{file}')
#         triseg = funclib.convert_back(seg, to='ternary')
#         np.save(f'{dir}seg_images_ternary/val/{file}', triseg)